# Compare top interests
In which we compare the top interests according to some old audience counts to determine the right cutoff for an initial interest analysis of US versus Mexico.

In [1]:
import pandas as pd

## Load data

In [2]:
interest_data = pd.read_csv('../../data/query_results/US_MX_native_interests_top_300.tsv', sep='\t', index_col=False)
interest_data.head()


,name,interests,ages_ranges,genders,behavior,scholarities,languages,family_statuses,relationship_statuses,geo_locations,household_composition,all_fields,targeting,response,dau_audience,mau_audience,timestamp,publisher_platforms
0,Broad list of interest per country,"{""or"": [6002839660079], ""name"": [""Cosmetics""]}","{""max"": 65, ""min"": 18}",0.0,"{""not"": [6015559470583, 6025978286383], ""name""...",NaN,NaN,NaN,NaN,"{""values"": [""US""], ""location_types"": [""home""],...",NaN,"(('interests', {u'or': [6002839660079L], u'nam...","{""flexible_spec"": [{""interests"": [{""id"": 60028...","{""data"":[{""bid_estimate"":{""min_bid"":186,""media...",36476999,42000000,1527244172,"[""facebook""]"
1,Broad list of interest per country,"{""or"": [6002839660079], ""name"": [""Cosmetics""]}","{""max"": 65, ""min"": 18}",0.0,"{""not"": [6015559470583, 6025978286383], ""name""...",NaN,NaN,NaN,NaN,"{""values"": [""MX""], ""location_types"": [""home""],...",NaN,"(('interests', {u'or': [6002839660079L], u'nam...","{""flexible_spec"": [{""interests"": [{""id"": 60028...","{""data"":[{""bid_estimate"":{""min_bid"":24,""median...",12341999,15000000,1527244172,"[""facebook""]"
2,Broad list of interest per country,"{""or"": [6002866718622], ""name"": [""Science""]}","{""max"": 65, ""min"": 18}",0.0,"{""not"": [6015559470583, 6025978286383], ""name""...",NaN,NaN,NaN,NaN,"{""values"": [""US""], ""location_types"": [""home""],...",NaN,"(('interests', {u'or': [6002866718622L], u'nam...","{""flexible_spec"": [{""interests"": [{""id"": 60028...","{""data"":[{""bid_estimate"":{""min_bid"":181,""media...",33333599,38000000,1527244172,"[""facebook""]"
3,Broad list of interest per country,"{""or"": [6002866718622], ""name"": [""Science""]}","{""max"": 65, ""min"": 18}",0.0,"{""not"": [6015559470583, 6025978286383], ""name""...",NaN,NaN,NaN,NaN,"{""values"": [""MX""], ""location_types"": [""home""],...",NaN,"(('interests', {u'or': [6002866718622L], u'nam...","{""flexible_spec"": [{""interests"": [{""id"": 60028...","{""data"":[{""bid_estimate"":{""min_bid"":25,""median...",9114599,11000000,1527244172,"[""facebook""]"
4,Broad list of interest per country,"{""or"": [6002867432822], ""name"": [""Beauty""]}","{""max"": 65, ""min"": 18}",0.0,"{""not"": [6015559470583, 6025978286383], ""name""...",NaN,NaN,NaN,NaN,"{""values"": [""US""], ""location_types"": [""home""],...",NaN,"(('interests', {u'or': [6002867432822L], u'nam...","{""flexible_spec"": [{""interests"": [{""id"": 60028...","{""data"":[{""bid_estimate"":{""min_bid"":181,""media...",64280799,76000000,1527244172,"[""facebook""]"
